In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
# !pip install --upgrade transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df = pd.read_csv('train.tsv', sep='\t')

# df = pd.read_csv('Privacy_classifier_review_sentiment.csv')
df = pd.read_csv('/content/drive/MyDrive/SENTIMENT MODEL/Sentiment Classifier TF-BERT22/Privacy_classifier_review_sentiment.csv')

sentimentmap={'positive':1, 'negative':0}

df["Sentiment"]=df['Sentiment'].map(sentimentmap)


df.head()

,Unnamed: 0,Learner Perspective on Privacy,Sentiment
0,0,One of the other reviewers has mentioned that ...,1
1,1,A wonderful little production. <br /><br />The...,1
2,2,I thought this was a wonderful way to spend ti...,1
3,3,Basically there's a family where a little boy ...,0
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['Sentiment']
df['Learner Perspective on Privacy']

df2=df[['Learner Perspective on Privacy', 'Sentiment']]
df2

,Learner Perspective on Privacy,Sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
80995,Privacy-related shortcomings on this platform ...,0
80996,"In a recent privacy breach, I was alarmed to f...",1
80997,The platform's privacy measures reflect a genu...,1
80998,"During my recent travels, I became increasingl...",1


### **Data Preparation**

In [ ]:
# df.drop(118202, inplace=True)
df2['Sentiment'] = df2['Sentiment'].astype(int)

<ipython-input-6-52ac6053c598>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Sentiment'] = df2['Sentiment'].astype(int)


In [ ]:
df2['Sentiment'].value_counts()

# '''
# The sentiment labels are:
# 0 - negative
# 1 - positive
# '''

,count
Sentiment,
1,40710
0,40290


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
token = tokenizer.encode_plus(
    df2['Learner Perspective on Privacy'].iloc[0],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
token
print (token.input_ids)
print(token.attention_mask)


tf.Tensor(
[[  101  1448  1104  1103  1168 19475  1144  3025  1115  1170  2903  1198
    122 16075  2004  1128   112  1325  1129 14111   119  1220  1132  1268
    117  1112  1142  1110  2839  1184  2171  1114  1143   119   133  9304
    120   135   133  9304   120   135  1109  1148  1645  1115  4168  1143
   1164 16075  1108  1157 28024  1105  8362  2087  2836  7520  4429  1104
   4289   117  1134  1383  1107  1268  1121  1103  1937 27157   119  4623
   1143   117  1142  1110  1136   170  1437  1111  1103  7859 21898  1137
    189  4060  2386   119  1188  1437  7561  1185 19739  1114 12747  1106
   5557   117  2673  1137  4289   119  2098  1110 16883   117  1107  1103
   5263  1329  1104  1103  1937   119   133  9304   120   135   133  9304
    120   135  1135  1110  1270   152  5301  1112  1115  1110  1103  8002
   1549  1106  1103 18638 13878  4354  1426 23544  3150 17103   119  1135
   7203  2871  1113 24464  1392   117  1126  6700  2237  1104  1103  3315
   1187  1155  1103  3652  

In [ ]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))
X_attn_masks.shape

(81000, 256)

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate( df2['Learner Perspective on Privacy'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [82]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [83]:
labels = np.zeros((len(df2), 2))
labels.shape

(81000, 2)

In [84]:
labels[np.arange(len(df2)), df2['Sentiment'].values] = 1 # one-hot encoded target tensor

In [85]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [86]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [87]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [88]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset

In [89]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [90]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [91]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 2), dtype=tf.float64, name=None))>

In [92]:
p = 0.9
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [93]:
train_size

4555

In [94]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

### **Model**

In [95]:
from transformers import TFBertModel

In [96]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [103]:
# Define input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Get BERT embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1]  # Using pooled output layer (2D)

# Intermediate dense layer with ReLU activation
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)

# Add dropout layer to prevent overfitting
dropout_layer_1 = tf.keras.layers.Dropout(0.3, name='dropout_layer_1')(intermediate_layer)

# Add another dense layer
additional_dense_layer = tf.keras.layers.Dense(256, activation='relu', name='additional_dense_layer')(dropout_layer_1)

# Add batch normalization layer
batch_norm_layer = tf.keras.layers.BatchNormalization(name='batch_norm_layer')(additional_dense_layer)

# Add another dropout layer
dropout_layer_2 = tf.keras.layers.Dropout(0.3, name='dropout_layer_2')(batch_norm_layer)

# Output layer with softmax activation
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(dropout_layer_2)

# Define the model
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)

# Print the model summary
sentiment_model.summary()

ValueError: Exception encountered when calling layer 'bert' (type TFBertMainLayer).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'bert' (type TFBertMainLayer):
  • input_ids=<KerasTensor shape=(None, 256), dtype=int32, sparse=None, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 256), dtype=int32, sparse=None, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [98]:
# Define input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
attn_masks
input_ids

<KerasTensor shape=(None, 256), dtype=int32, sparse=None, name=input_ids>

In [99]:
import keras
print(keras.__version__)

3.4.1


In [102]:
from transformers import TFBertModel
import tensorflow as tf

# Load the BERT model with pretrained weights
model = TFBertModel.from_pretrained('bert-base-cased')

# Define input layers for input_ids and attention_mask
input_ids = tf.keras.layers.Input(shape=(None,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(None,), name='attention_mask', dtype='int32')

# Get the outputs from BERT
bert_output = model(input_ids, attention_mask=attn_masks)  # Call the model directly

# Use the pooled output (the second element of the output tuple)
bert_embds = bert_output[1]  # Using pooled output layer

# Adding additional layers on top of the BERT output
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer)

# Creating the model
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_10' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_10' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, None), dtype=int32, sparse=None, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, None), dtype=int32, sparse=None, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
# Get BERT embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1]  # Using pooled output layer (2D)

# Intermediate dense layer with ReLU activation
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)

# Add dropout layer to prevent overfitting
dropout_layer_1 = tf.keras.layers.Dropout(0.3, name='dropout_layer_1')(intermediate_layer)

# Add another dense layer
additional_dense_layer = tf.keras.layers.Dense(256, activation='relu', name='additional_dense_layer')(dropout_layer_1)

# Add batch normalization layer
batch_norm_layer = tf.keras.layers.BatchNormalization(name='batch_norm_layer')(additional_dense_layer)

# Add another dropout layer
dropout_layer_2 = tf.keras.layers.Dropout(0.3, name='dropout_layer_2')(batch_norm_layer)

# Output layer with softmax activation
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(dropout_layer_2)

# Define the model
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)

# Print the model summary
sentiment_model.summary()

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall(thresholds=0)


# f1_score = 2 * (precision * recall) / (precision + recall)

# print(f1_score)

In [ ]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc,precision,recall])

# Training The Model

In [ ]:
hist = sentiment_model.fit(  train_dataset,
    validation_data=val_dataset,
    epochs=10
)

**MODEL EVALUATION**

**Plot ROC Curve**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import tensorflow as tf


# Step 1: Extract True Labels
def extract_labels(dataset):
    y_true = []
    for batch in dataset:
        _, labels = batch
        y_true.extend(labels.numpy())  # Convert labels to numpy array and extend the list
    return np.array(y_true)

# Extract true labels
y_true = extract_labels(val_dataset)

# Convert to binary labels if they are one-hot encoded
if y_true.ndim > 1 and y_true.shape[1] > 1:
    y_true = np.argmax(y_true, axis=1)

# Step 2: Make Predictions
y_pred_proba = sentiment_model.predict(val_dataset)

# If y_pred_proba is a 2D array with probabilities for each class,
# make sure to select the probability for the positive class
# For binary classification, assume positive class is index 1
if y_pred_proba.shape[1] > 1:
    y_pred_proba = y_pred_proba[:, 1]  # Get probabilities for the positive class
else:
    y_pred_proba = y_pred_proba.flatten()  # If output is already single probability per sample

# Step 3: Compute ROC Curve Data
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)

# Compute AUC
roc_auc = auc(fpr, tpr)

# Step 4: Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# # Splitting the dataset
# train_dataset = dataset.take(train_size)
# val_dataset = dataset.skip(train_size)

# # Training the model
# hist = sentiment_model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=3
# )


**Ploting the Loss and Accuracy on the Epochs**

In [ ]:
# hist.history['accuracy']

import matplotlib.pyplot as plt
hist.history['val_accuracy']

# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

**PLOT THE ACCURACY, PRECISION ETC**


In [ ]:
# Extract the metrics from history
history = hist.history
# Define the metrics you want to tabulate
metrics = ['loss', 'accuracy', 'precision', 'recall']
# Create a DataFrame to store the metrics
data = {
    "Metric": metrics,
    "Training Set": [history[metric][-1] for metric in metrics],  # Last epoch training values
    "Validation Set": [history[f"val_{metric}"][-1] for metric in metrics]  # Last epoch validation values
}
# Create the DataFrame
df = pd.DataFrame(data)
# styled_df
# Display the table
df


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Step 1: Obtain the true labels (convert from one-hot encoding if necessary)
y_true = np.concatenate([y for x, y in val_dataset], axis=0)  # True labels from validation data
if y_true.ndim > 1 and y_true.shape[1] > 1:  # Check if one-hot encoded
    y_true = np.argmax(y_true, axis=1)  # Convert to single-label format
# Step 2: Obtain the predicted labels
y_pred_probs = sentiment_model.predict(val_dataset)  # Predict probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Get the predicted class
# Step 3: Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Step 4: Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
sentiment_model.save('sentiment_model')

**CASSCADED MODEL**

*Privacy classifier configuration*

In [ ]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)

In [ ]:
from torch.utils.data import Dataset
import torch, os

In [ ]:
# sentiment_model = tf.keras.models.load_model('sentiment_model')
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

privacy_model = "/content/drive/MyDrive/PRIVACY MODEL/PRIVACY MODEL/BERT_for_Multi_Class_Classification (Privacy_Original)/Model/tmp-checkpoint-500"

device = 0 if torch.cuda.is_available() else -1
model = BertForSequenceClassification.from_pretrained(privacy_model)
# tokenizer= BertTokenizerFast.from_pretrained(privacy_model)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)



In [ ]:
nlp("Within e-learning environments designed for children and adolescents, cybercriminals may capitalize on security weaknesses to obtain sensitive personal information, such as birthdates and school affiliations, paving the way for child identity theft. The repercussions of data breaches on minors privacy can be severe, causing long-term harm to victims and their families")


In [ ]:
print(os.listdir(privacy_model))

### **Prediction**

In [ ]:
sentiment_model = tf.keras.models.load_model('sentiment_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }
def make_prediction(model, processed_data, classes=['Negative', 'Positive']):
    probs = model.predict(processed_data)[0]
    return np.argmax(probs)

In [ ]:
input_text = input('Describe privacy issues: ')
processed_data = prepare_data(input_text, tokenizer)

result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

# Save Responses

In [ ]:
import pandas as pd

# Function to collect user input
def get_user_input():
    # user_id = input("Enter User ID: ")
    input_text = input("Enter user input: ")
    # privacy_issue = input("Enter privacy issues (if any): ")
    return input_text
# Prompt user for input
user_input_data = []
# while True:
input_text= get_user_input()
# Perform sentiment prediction here (you need to implement this)
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
# sentiment_prediction = predict_sentiment(user_input)
user_input_data.append({
    'InputText': input_text,
    'Sentiment': result
})

    # Check if the user wants to enter more data
    # more_data = input("Do you want to enter more data? (yes/no): ")
    # if more_data.lower() != 'yes':
    #     break
# Create DataFrame
# df = pd.DataFrame(user_input_data)
# Create DataFrame from new user input data
new_df = pd.DataFrame(user_input_data)

# Load existing data from Excel file (if exists)
try:
    existing_df = pd.read_excel('results.xlsx')
except FileNotFoundError:
    existing_df = pd.DataFrame()

# Concatenate new data with existing data
combined_df = pd.concat([existing_df, new_df], ignore_index=True)

# Save combined data to Excel file
combined_df.to_excel('results.xlsx', index=False)
# # Save DataFrame to Excel
# df.to_excel('results.xlsx', index=False)

print(result)


In [ ]:
#Privacy considerations are lacking, creating unease in my learning experience

In [ ]:

# Create DataFrame from new user input data
new_df = pd.DataFrame(user_input_data)

# Load existing data from Excel file (if exists)
try:
    existing_df = pd.read_excel('sentiments.xlsx')
except FileNotFoundError:
    existing_df = pd.DataFrame()

# Concatenate new data with existing data
combined_df = pd.concat([existing_df, new_df], ignore_index=True)

# Save combined data to Excel file
combined_df.to_excel('sentiments.xlsx', index=False)

# Save responses

In [ ]:
import pandas as pd


# Test the model with new data
user_inputs = [
    "I do not love using this product!",
    "I'm not satisfied with the customer service.",
    "This app is amazing and easy to use.",
    # Add more user inputs as needed
    ]

data = {"ID": [], "Sentiment": []}

for idx, input_text in enumerate(user_inputs, start=1):
    sentiment = predict_sentiment(input_text)
    data["ID"].append(idx)
    data["Sentiment"].append(sentiment)
    print(f"Input: {input_text}\nPredicted Sentiment: {sentiment}\n")

    # SAVE DATA AS CSV
df = pd.DataFrame(data)

# Save DataFrame to Excel
df.to_excel('predictions.xlsx', index=False)
print("Data appended to predictions.xlsx")


In [ ]:

# # Test the model with new data
# user_inputs = [
#     "I do not love using this product!",
#     "I'm not satisfied with the customer service.",
#     "This app is amazing and easy to use.",
#     # Add more user inputs as needed
#     ]

# data = {"ID": [], "Sentiment": []}

# for idx, input_text in enumerate(user_inputs, start=1):
#     sentiment = predict_sentiment(input_text)
#     data["ID"].append(idx)
#     data["Sentiment"].append(sentiment)
#     print(f"Input: {input_text}\nPredicted Sentiment: {sentiment}\n")

#     # SAVE DATA AS CSV
# df = pd.DataFrame(data)

# # Save DataFrame to Excel
# df.to_excel('predictions.xlsx', index=False)
# print("Data appended to predictions.xlsx")
